einops的强项是把张量维度操作具象化。
例如:
```
from einops import rearrange
output_tesnor = rearrange(input_tensor,'h w c -> c h w')
```

In [13]:
# 读入图片
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
img = np.array(Image.open('./data/images/001.png'))/ 255.
img.shape, img.dtype

((808, 958, 3), dtype('float64'))

In [4]:
# 载入einops
from einops import rearrange, reduce, repeat

In [5]:
#调换顺序
out = rearrange(img, 'h w c -> w h c')
out.shape, out.dtype

((958, 808, 3), dtype('float64'))

In [14]:
#相较于pytorch中的permute更高级的玩法:将某维度看作两种维度的乘，给出值，直接拆分
import torch
a = torch.randn(3,9,9)
output = rearrange(a, 'c (r p) w -> c r p w', p = 3)
print(output.shape)

torch.Size([3, 3, 3, 9])


In [20]:
# reduce操作
a = torch.randn(3, 8, 8)
output_tensor = reduce(a, 'c (h h2) (w w2) -> h w c', 'mean', h2=2, w2=2)
output_tensor2 = reduce(a, 'h w c -> h w', 'mean') # 约简运算1
output_tensor3 = reduce(a, 'h w c -> h w 1', 'mean') # 约简运算2
output_tensor.shape

torch.Size([4, 4, 3])

In [24]:
# repeat操作：指定复制的层数
a = torch.randn(9, 9)
output_tensor = repeat(a, 'h w -> c h w', c= 3)
output_tensor.shape

torch.Size([3, 9, 9])

# 另外，einops可以直接嵌套在pytorch的layer层中

In [ ]:
# example given for pytorch, but code in other frameworks is almost identical
from torch.nn import Sequential, Conv2d, MaxPool2d, Linear, ReLU
from einops.layers.torch import Rearrange

model = Sequential(
    Conv2d(3, 6, kernel_size=5),
    MaxPool2d(kernel_size=2),
    Conv2d(6, 16, kernel_size=5),
    MaxPool2d(kernel_size=2),
    # flattening
    Rearrange('b c h w -> b (c h w)'),
    Linear(16*5*5, 120),
    ReLU(),
    Linear(120, 10),
)